In [1]:
import pandas as pd
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt

In [2]:
!pip install ffmpeg-python

  Using cached ffmpeg_python-0.2.0-py3-none-any.whl (25 kB)
     ---------------------------------------- 0.0/840.9 kB ? eta -:--:--
      --------------------------------------- 20.5/840.9 kB ? eta -:--:--
     -- ---------------------------------- 51.2/840.9 kB 650.2 kB/s eta 0:00:02
     --- --------------------------------- 71.7/840.9 kB 558.5 kB/s eta 0:00:02
     --- --------------------------------- 71.7/840.9 kB 558.5 kB/s eta 0:00:02
     --- --------------------------------- 71.7/840.9 kB 558.5 kB/s eta 0:00:02
     --- --------------------------------- 71.7/840.9 kB 558.5 kB/s eta 0:00:02
     --- --------------------------------- 71.7/840.9 kB 558.5 kB/s eta 0:00:02
     --- --------------------------------- 71.7/840.9 kB 558.5 kB/s eta 0:00:02
     --- --------------------------------- 71.7/840.9 kB 558.5 kB/s eta 0:00:02
     --- --------------------------------- 71.7/840.9 kB 558.5 kB/s eta 0:00:02
     --- --------------------------------- 71.7/840.9 kB 558.5 kB/s eta 0

In [3]:
def process_video15(video_file_path, num_frames=250):
    # Open the video file
    cap = cv2.VideoCapture(video_file_path)

    # Read the first frame
    ret, frame = cap.read()

    # Collect the frames at random positions
    frame_indices = np.random.randint(0, cap.get(cv2.CAP_PROP_FRAME_COUNT), num_frames)
    frames = np.empty((num_frames, *frame.shape), dtype=frame.dtype)
    for i, idx in enumerate(frame_indices):
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        frames[i] = frame

    # Apply median filtering to the collected frames
    result = np.median(frames, axis=0).astype(dtype=np.uint8)

    # Calculate the average intensity of the left and right halves of the median frame
    height, width, _ = result.shape
    left_half_avg = np.mean(result[:, :width // 2])
    right_half_avg = np.mean(result[:, width // 2:])

    # Determine if video is flipped or not based on average intensities
    is_flipped = right_half_avg < left_half_avg

    # Release the video file and close all windows
    cap.release()

    return is_flipped

In [4]:
video_dir = r'C:\Users\andri\video'

for filename in os.listdir(video_dir):
    if filename.endswith('.avi'):
        video_file_path = os.path.join(video_dir, filename)
        is_flipped = process_video15(video_file_path)
        if is_flipped.any():
            print(f'{filename} is flipped')
        else:
            print(f'{filename} is not flipped')
            
            

20220928-154222-C16980_22-5_SIT_SDS_AM22043_day11__woICR__000.avi is not flipped
20220928-154222-C16980_22-5_SIT_SDS_AM22044_day11__woICR__000.avi is flipped
20220928-154700-C16980_22-5_SIT_SDS_AM22043_day11__wICR__000.avi is not flipped
20220928-154700-C16980_22-5_SIT_SDS_AM22044_day11__wICR__000.avi is flipped
20220928-161310-C16980_22-5_SIT_SDS_AM22045_day11_woICR__000.avi is not flipped
20220928-161310-C16980_22-5_SIT_SDS_AM22046_day11_woICR__000.avi is flipped
20220928-161730-C16980_22-5_SIT_SDS_AM22045_day11_wICR__000.avi is not flipped
20220928-161730-C16980_22-5_SIT_SDS_AM22046_day11_wICR__000.avi is flipped


In [8]:
import os
import json
# import subprocess
import ffmpeg

# ... (your existing code)

# def flip_video(video_file_path):
#     flipped_video_file_path = os.path.splitext(video_file_path)[0] + "_flipped.avi"
#     subprocess.run(["ffmpeg", "-i", video_file_path, "-vf", "hflip", flipped_video_file_path])
#     return flipped_video_file_path


def flip_video(video_file_path):
    flipped_video_file_path = os.path.splitext(video_file_path)[0] + "_flipped.avi"

    # Use ffmpeg-python instead of subprocess to call ffmpeg with specified options
    stream = ffmpeg.input(video_file_path)
    stream = ffmpeg.hflip(stream)
    stream = ffmpeg.output(stream, flipped_video_file_path, format="avi", vcodec="h264_nvenc", qscale=0)
    ffmpeg.run(stream)

    return flipped_video_file_path

def create_json_file(file_path, data):
    with open(file_path, 'w') as json_file:
        json.dump(data, json_file)

root_dir = r"C:\Users\andri\PycharmProjects\testvideo"

for dirpath, dirnames, filenames in os.walk(root_dir):
    if 'raw_video' in dirnames:
        raw_video_dir = os.path.join(dirpath, 'raw_video')
        for filename in os.listdir(raw_video_dir):
            if filename.endswith('.avi'):
                video_file_path = os.path.join(raw_video_dir, filename)
                is_flipped = process_video15(video_file_path)
                
                # Cast numpy boolean to native Python boolean
                is_flipped = bool(is_flipped)
                json_data = {"is_flipped": is_flipped}

                if is_flipped:
                    print(f'{filename} is flipped')
                    flipped_video_file_path = flip_video(video_file_path)

                    # Move the original video to the original_video folder
                    original_video_dir = os.path.join(raw_video_dir, "original_video")
                    os.makedirs(original_video_dir, exist_ok=True)
                    os.rename(video_file_path, os.path.join(original_video_dir, filename))

                    # Rename the flipped video to the original file name
                    os.rename(flipped_video_file_path, video_file_path)
                else:
                    print(f'{filename} is not flipped')

                # Create a JSON file with is_flipped information
                json_file_path = os.path.join(raw_video_dir, os.path.splitext(filename)[0] + '.json')
                create_json_file(json_file_path, json_data)


20220901-172740-C16980_22-5_SIT_SDS_AM22033_day11_wICR__000.avi is not flipped
20220901-172037-C16980_22-5_SIT_SDS_AM22033_day11_woICR__000.avi is not flipped
20220908-161032-C16980_22-5_SIT_SDS_AM22035_day11_wICR__000.avi is not flipped
20220908-160544-C16980_22-5_SIT_SDS_AM22035_day11_woICR__000.avi is not flipped
20220908-161032-C16980_22-5_SIT_SDS_AM22036_day11_wICR__000.avi is flipped
20220908-160544-C16980_22-5_SIT_SDS_AM22036_day11_woICR__000.avi is flipped
20220928-154700-C16980_22-5_SIT_SDS_AM22043_day11__wICR__000.avi is not flipped
20220928-154222-C16980_22-5_SIT_SDS_AM22043_day11__woICR__000.avi is not flipped
20220928-154700-C16980_22-5_SIT_SDS_AM22044_day11__wICR__000.avi is flipped
20220928-154222-C16980_22-5_SIT_SDS_AM22044_day11__woICR__000.avi is flipped
20220928-161730-C16980_22-5_SIT_SDS_AM22045_day11_wICR__000.avi is not flipped
20220928-161310-C16980_22-5_SIT_SDS_AM22045_day11_woICR__000.avi is not flipped
20220928-161730-C16980_22-5_SIT_SDS_AM22046_day11_wICR__00

In [9]:
def process_video18(video_file_path, num_frames=100):
    cap = cv2.VideoCapture(video_file_path)

    frame_indices = np.random.randint(0, cap.get(cv2.CAP_PROP_FRAME_COUNT), num_frames)
    left_half_sum, right_half_sum = 0, 0

    for idx in frame_indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        left_half_sum += np.mean(frame[:, :frame.shape[1] // 2])
        right_half_sum += np.mean(frame[:, frame.shape[1] // 2:])

    cap.release()

    left_half_avg, right_half_avg = left_half_sum / num_frames, right_half_sum / num_frames
    is_flipped = right_half_avg < left_half_avg

    return is_flipped


In [10]:
video_dir = r'C:\Users\andri\video'

for filename in os.listdir(video_dir):
    if filename.endswith('.avi'):
        video_file_path = os.path.join(video_dir, filename)
        is_flipped = process_video18(video_file_path)
        if is_flipped.any():
            print(f'{filename} is flipped')
        else:
            print(f'{filename} is not flipped')

20220928-154222-C16980_22-5_SIT_SDS_AM22043_day11__woICR__000.avi is not flipped
20220928-154222-C16980_22-5_SIT_SDS_AM22044_day11__woICR__000.avi is flipped
20220928-154700-C16980_22-5_SIT_SDS_AM22043_day11__wICR__000.avi is not flipped
20220928-154700-C16980_22-5_SIT_SDS_AM22044_day11__wICR__000.avi is flipped
20220928-161310-C16980_22-5_SIT_SDS_AM22045_day11_woICR__000.avi is not flipped
20220928-161310-C16980_22-5_SIT_SDS_AM22046_day11_woICR__000.avi is flipped
20220928-161730-C16980_22-5_SIT_SDS_AM22045_day11_wICR__000.avi is not flipped
20220928-161730-C16980_22-5_SIT_SDS_AM22046_day11_wICR__000.avi is flipped


# Checking function performance

In [5]:
import numpy as np
import cv2

def process_video20(video_file_path, num_frames=100):
    cap = cv2.VideoCapture(video_file_path)

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_indices = np.random.randint(0, total_frames, num_frames)

    height, width, _ = (int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)),
                        int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
                        int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))

    left_half_sum = np.zeros((height, width // 2))
    right_half_sum = np.zeros((height, width // 2))

    for idx in frame_indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        left_half_sum += gray_frame[:, :width // 2]
        right_half_sum += gray_frame[:, width // 2:]

    cap.release()

    left_half_avg = np.mean(left_half_sum) / num_frames
    right_half_avg = np.mean(right_half_sum) / num_frames

    is_flipped = right_half_avg < left_half_avg

    return is_flipped



In [6]:
video_dir = r'C:\Users\andri\video'

for filename in os.listdir(video_dir):
    if filename.endswith('.avi'):
        video_file_path = os.path.join(video_dir, filename)
        is_flipped = process_video20(video_file_path)
        if is_flipped.any():
            print(f'{filename} is flipped')
        else:
            print(f'{filename} is not flipped')

20220928-154222-C16980_22-5_SIT_SDS_AM22043_day11__woICR__000.avi is not flipped
20220928-154222-C16980_22-5_SIT_SDS_AM22044_day11__woICR__000.avi is flipped
20220928-154700-C16980_22-5_SIT_SDS_AM22043_day11__wICR__000.avi is not flipped
20220928-154700-C16980_22-5_SIT_SDS_AM22044_day11__wICR__000.avi is flipped
20220928-161310-C16980_22-5_SIT_SDS_AM22045_day11_woICR__000.avi is not flipped
20220928-161310-C16980_22-5_SIT_SDS_AM22046_day11_woICR__000.avi is flipped
20220928-161730-C16980_22-5_SIT_SDS_AM22045_day11_wICR__000.avi is not flipped
20220928-161730-C16980_22-5_SIT_SDS_AM22046_day11_wICR__000.avi is flipped


# Checking new function

In [7]:
import numpy as np
import cv2

def process_video33(video_file_path, num_frames=100):
    cap = cv2.VideoCapture(video_file_path)

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_indices = np.random.randint(0, total_frames, num_frames)

    height, width, _ = (int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)),
                        int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
                        int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))

    left_half_sum = 0
    right_half_sum = 0

    width_half = width // 2

    for idx in frame_indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        left_half_sum += np.sum(gray_frame[:, :width_half])
        right_half_sum += np.sum(gray_frame[:, width_half:])

    cap.release()

    left_half_avg = left_half_sum / (num_frames * height * width_half)
    right_half_avg = right_half_sum / (num_frames * height * width_half)

    is_flipped = right_half_avg < left_half_avg

    return is_flipped


In [8]:
video_dir = r'C:\Users\andri\video'

for filename in os.listdir(video_dir):
    if filename.endswith('.avi'):
        video_file_path = os.path.join(video_dir, filename)
        is_flipped = process_video33(video_file_path)
        if is_flipped.any():
            print(f'{filename} is flipped')
        else:
            print(f'{filename} is not flipped')

20220928-154222-C16980_22-5_SIT_SDS_AM22043_day11__woICR__000.avi is not flipped
20220928-154222-C16980_22-5_SIT_SDS_AM22044_day11__woICR__000.avi is flipped
20220928-154700-C16980_22-5_SIT_SDS_AM22043_day11__wICR__000.avi is not flipped
20220928-154700-C16980_22-5_SIT_SDS_AM22044_day11__wICR__000.avi is flipped
20220928-161310-C16980_22-5_SIT_SDS_AM22045_day11_woICR__000.avi is not flipped
20220928-161310-C16980_22-5_SIT_SDS_AM22046_day11_woICR__000.avi is flipped
20220928-161730-C16980_22-5_SIT_SDS_AM22045_day11_wICR__000.avi is not flipped
20220928-161730-C16980_22-5_SIT_SDS_AM22046_day11_wICR__000.avi is flipped


In [11]:
import numpy as np
import cv2

def process_video55(video_file_path, num_frames=100):
    # Open the video file
    cap = cv2.VideoCapture(video_file_path)

    # Get the total number of frames in the video
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    # Generate random frame indices to process
    frame_indices = np.random.randint(0, total_frames, num_frames)

    # Get the height and width of the video as a NumPy array
    frame_dimensions = np.array([int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)),
                                  int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))])

    # Calculate the midpoint of the frame width
    width_half = frame_dimensions[1] // 2

    # Initialize the average brightness values for the left and right halves of the frames
    left_half_avg = 0
    right_half_avg = 0

    # Process each randomly selected frame
    for idx in frame_indices:
        # Set the current frame to be read
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        # Read the frame from the video
        ret, frame = cap.read()
        # Convert the frame to grayscale
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # Update the average brightness of the left and right halves of the frames
        left_half_avg += np.sum(gray_frame[:, :width_half]) / (frame_dimensions[0] * width_half)
        right_half_avg += np.sum(gray_frame[:, width_half:]) / (frame_dimensions[0] * width_half)

    # Close the video file
    cap.release()

    # Calculate the final average brightness values for the left and right halves of the frames
    left_half_avg /= num_frames
    right_half_avg /= num_frames

    # Determine if the video is flipped based on the average brightness of the halves
    is_flipped = right_half_avg < left_half_avg

    # Return whether the video is flipped
    return is_flipped


In [10]:
video_dir = r'C:\Users\andri\video'

for filename in os.listdir(video_dir):
    if filename.endswith('.avi'):
        video_file_path = os.path.join(video_dir, filename)
        is_flipped = process_video55(video_file_path)
        if is_flipped.any():
            print(f'{filename} is flipped')
        else:
            print(f'{filename} is not flipped')

20220928-154222-C16980_22-5_SIT_SDS_AM22043_day11__woICR__000.avi is not flipped
20220928-154222-C16980_22-5_SIT_SDS_AM22044_day11__woICR__000.avi is flipped
20220928-154700-C16980_22-5_SIT_SDS_AM22043_day11__wICR__000.avi is not flipped
20220928-154700-C16980_22-5_SIT_SDS_AM22044_day11__wICR__000.avi is flipped
20220928-161310-C16980_22-5_SIT_SDS_AM22045_day11_woICR__000.avi is not flipped
20220928-161310-C16980_22-5_SIT_SDS_AM22046_day11_woICR__000.avi is flipped
20220928-161730-C16980_22-5_SIT_SDS_AM22045_day11_wICR__000.avi is not flipped
20220928-161730-C16980_22-5_SIT_SDS_AM22046_day11_wICR__000.avi is flipped


# Checking 250 video frames

In [18]:
def process_video22(video_file_path, num_frames=250):
    # Open the video file
    cap = cv2.VideoCapture(video_file_path)

    # Read the first frame
    ret, frame = cap.read()

    # Collect the frames at random positions
    frame_indices = np.random.randint(0, cap.get(cv2.CAP_PROP_FRAME_COUNT), num_frames)
    frames = np.empty((num_frames, *frame.shape), dtype=frame.dtype)
    for i, idx in enumerate(frame_indices):
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        frames[i] = frame

    # Apply median filtering to the collected frames
    result = np.median(frames, axis=0).astype(dtype=np.uint8)

    # Calculate the average intensity of the left and right halves of the median frame
    height, width, _ = result.shape
    left_half_avg = np.mean(result[:, :width // 2])
    right_half_avg = np.mean(result[:, width // 2:])

    # Determine if video is flipped or not based on average intensities
    is_flipped = right_half_avg < left_half_avg

    # Release the video file and close all windows
    cap.release()

    return is_flipped

In [19]:
import os
import json
# import subprocess
import ffmpeg

# ... (your existing code)

# def flip_video(video_file_path):
#     flipped_video_file_path = os.path.splitext(video_file_path)[0] + "_flipped.avi"
#     subprocess.run(["ffmpeg", "-i", video_file_path, "-vf", "hflip", flipped_video_file_path])
#     return flipped_video_file_path


def flip_video(video_file_path):
    flipped_video_file_path = os.path.splitext(video_file_path)[0] + "_flipped.avi"

    # Use ffmpeg-python instead of subprocess to call ffmpeg with specified options
    stream = ffmpeg.input(video_file_path)
    stream = ffmpeg.hflip(stream)
    stream = ffmpeg.output(stream, flipped_video_file_path, format="avi", vcodec="h264_nvenc", qscale=0)
    ffmpeg.run(stream)

    return flipped_video_file_path

def create_json_file(file_path, data):
    with open(file_path, 'w') as json_file:
        json.dump(data, json_file)
        
root_dir = r'C:\Users\andri\PycharmProjects\testvideo2'
        
for dirpath, dirnames, filenames in os.walk(root_dir):
    if 'raw_video' in dirnames:
        raw_video_dir = os.path.join(dirpath, 'raw_video')
        for filename in os.listdir(raw_video_dir):
            if filename.endswith('.avi'):
                video_file_path = os.path.join(raw_video_dir, filename)
                try:
                    is_flipped = process_video22(video_file_path)

                    # Cast numpy boolean to native Python boolean
                    is_flipped = bool(is_flipped)
                    json_data = {"is_flipped": is_flipped}

                    if is_flipped:
                        print(f'{filename} is flipped')
                        flipped_video_file_path = flip_video(video_file_path)

                        # Move the original video to the original_video folder
                        original_video_dir = os.path.join(raw_video_dir, "original_video")
                        os.makedirs(original_video_dir, exist_ok=True)
                        os.rename(video_file_path, os.path.join(original_video_dir, filename))

                        # Rename the flipped video to the original file name
                        os.rename(flipped_video_file_path, video_file_path)
                    else:
                        print(f'{filename} is not flipped')

                    # Create a JSON file with is_flipped information
                    json_file_path = os.path.join(raw_video_dir, os.path.splitext(filename)[0] + '.json')
                    create_json_file(json_file_path, json_data)

                except Exception as e:
                    print(f"{filename} does not contain any frames for analysis. Please check the video. Error: {e}")


20220901-172740-C16980_22-5_SIT_SDS_AM22033_day11_wICR__000.avi is not flipped
20220901-172037-C16980_22-5_SIT_SDS_AM22033_day11_woICR__000.avi is not flipped
20220908-161032-C16980_22-5_SIT_SDS_AM22035_day11_wICR__000.avi is not flipped
20220908-160544-C16980_22-5_SIT_SDS_AM22035_day11_woICR__000.avi is not flipped
20220908-161032-C16980_22-5_SIT_SDS_AM22036_day11_wICR__000.avi is flipped
20220908-160544-C16980_22-5_SIT_SDS_AM22036_day11_woICR__000.avi is flipped
20220928-154700-C16980_22-5_SIT_SDS_AM22043_day11__wICR__000.avi is not flipped
20220928-154222-C16980_22-5_SIT_SDS_AM22043_day11__woICR__000.avi is not flipped
20220928-154700-C16980_22-5_SIT_SDS_AM22044_day11__wICR__000.avi is flipped
20220928-154222-C16980_22-5_SIT_SDS_AM22044_day11__woICR__000.avi is flipped
20220928-161730-C16980_22-5_SIT_SDS_AM22045_day11_wICR__000.avi is not flipped
20220928-161310-C16980_22-5_SIT_SDS_AM22045_day11_woICR__000.avi is not flipped
20220928-161730-C16980_22-5_SIT_SDS_AM22046_day11_wICR__00

# Test new function

In [ ]:
import numpy as np
import cv2

def process_video55(video_file_path, num_frames=100):
    # Open the video file
    cap = cv2.VideoCapture(video_file_path)

    # Get the total number of frames in the video
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    # Generate random frame indices to process
    frame_indices = np.random.randint(0, total_frames, num_frames)

    # Get the height and width of the video as a NumPy array
    frame_dimensions = np.array([int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)),
                                  int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))])

    # Calculate the midpoint of the frame width
    width_half = frame_dimensions[1] // 2

    # Initialize the average brightness values for the left and right halves of the frames
    left_half_avg = 0
    right_half_avg = 0

    # Process each randomly selected frame
    for idx in frame_indices:
        # Set the current frame to be read
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        # Read the frame from the video
        ret, frame = cap.read()
        # Convert the frame to grayscale
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # Update the average brightness of the left and right halves of the frames
        left_half_avg += np.sum(gray_frame[:, :width_half]) / (frame_dimensions[0] * width_half)
        right_half_avg += np.sum(gray_frame[:, width_half:]) / (frame_dimensions[0] * width_half)

    # Close the video file
    cap.release()

    # Calculate the final average brightness values for the left and right halves of the frames
    left_half_avg /= num_frames
    right_half_avg /= num_frames

    # Determine if the video is flipped based on the average brightness of the halves
    is_flipped = right_half_avg < left_half_avg

    # Return whether the video is flipped
    return is_flipped

In [12]:
import os
import json
# import subprocess
import ffmpeg

# def flip_video(video_file_path):
#     flipped_video_file_path = os.path.splitext(video_file_path)[0] + "_flipped.avi"
#     subprocess.run(["ffmpeg", "-i", video_file_path, "-vf", "hflip", flipped_video_file_path])
#     return flipped_video_file_path


def flip_video(video_file_path):
    flipped_video_file_path = os.path.splitext(video_file_path)[0] + "_flipped.avi"

    # Use ffmpeg-python instead of subprocess to call ffmpeg with specified options
    stream = ffmpeg.input(video_file_path)
    stream = ffmpeg.hflip(stream)
    stream = ffmpeg.output(stream, flipped_video_file_path, format="avi", vcodec="h264_nvenc", qscale=0)
    ffmpeg.run(stream)

    return flipped_video_file_path

def create_json_file(file_path, data):
    with open(file_path, 'w') as json_file:
        json.dump(data, json_file)
        
root_dir = r'C:\Users\andri\PycharmProjects\test04022023'
        
for dirpath, dirnames, filenames in os.walk(root_dir):
    if 'raw_video' in dirnames:
        raw_video_dir = os.path.join(dirpath, 'raw_video')
        for filename in os.listdir(raw_video_dir):
            if filename.endswith('.avi'):
                video_file_path = os.path.join(raw_video_dir, filename)
                try:
                    is_flipped = process_video55(video_file_path)

                    # Cast numpy boolean to native Python boolean
                    is_flipped = bool(is_flipped)
                    json_data = {"is_flipped": is_flipped}

                    if is_flipped:
                        print(f'{filename} is flipped')
                        flipped_video_file_path = flip_video(video_file_path)

                        # Move the original video to the original_video folder
                        original_video_dir = os.path.join(raw_video_dir, "original_video")
                        os.makedirs(original_video_dir, exist_ok=True)
                        os.rename(video_file_path, os.path.join(original_video_dir, filename))

                        # Rename the flipped video to the original file name
                        os.rename(flipped_video_file_path, video_file_path)
                    else:
                        print(f'{filename} is not flipped')

                    # Create a JSON file with is_flipped information
                    json_file_path = os.path.join(raw_video_dir, os.path.splitext(filename)[0] + '.json')
                    create_json_file(json_file_path, json_data)

                except Exception as e:
                    print(f"{filename} does not contain any frames for analysis. Please check the video. Error: {e}")

20220901-172740-C16980_22-5_SIT_SDS_AM22033_day11_wICR__000.avi is not flipped
20220901-172037-C16980_22-5_SIT_SDS_AM22033_day11_woICR__000.avi is not flipped
20220908-161032-C16980_22-5_SIT_SDS_AM22035_day11_wICR__000.avi is not flipped
20220908-160544-C16980_22-5_SIT_SDS_AM22035_day11_woICR__000.avi is not flipped
20220908-161032-C16980_22-5_SIT_SDS_AM22036_day11_wICR__000.avi is flipped
20220908-160544-C16980_22-5_SIT_SDS_AM22036_day11_woICR__000.avi is flipped
20220928-154700-C16980_22-5_SIT_SDS_AM22043_day11__wICR__000.avi is not flipped
20220928-154222-C16980_22-5_SIT_SDS_AM22043_day11__woICR__000.avi is not flipped
20220928-154700-C16980_22-5_SIT_SDS_AM22044_day11__wICR__000.avi is flipped
20220928-154222-C16980_22-5_SIT_SDS_AM22044_day11__woICR__000.avi is flipped
20220928-161730-C16980_22-5_SIT_SDS_AM22045_day11_wICR__000.avi is not flipped
20220928-161310-C16980_22-5_SIT_SDS_AM22045_day11_woICR__000.avi is not flipped
20220928-161730-C16980_22-5_SIT_SDS_AM22046_day11_wICR__00

# Final function test (transform this code into separe .py file)

In [ ]:
import os
import json
# import subprocess
import ffmpeg

# def flip_video(video_file_path):
#     flipped_video_file_path = os.path.splitext(video_file_path)[0] + "_flipped.avi"
#     subprocess.run(["ffmpeg", "-i", video_file_path, "-vf", "hflip", flipped_video_file_path])
#     return flipped_video_file_path


def flip_video(video_file_path):
    flipped_video_file_path = os.path.splitext(video_file_path)[0] + "_flipped.avi"

    # Use ffmpeg-python instead of subprocess to call ffmpeg with specified options
    stream = ffmpeg.input(video_file_path)
    stream = ffmpeg.hflip(stream)
    stream = ffmpeg.output(stream, flipped_video_file_path, format="avi", vcodec="h264_nvenc", qscale=0)
    ffmpeg.run(stream)

    return flipped_video_file_path

def create_json_file(file_path, data):
    with open(file_path, 'w') as json_file:
        json.dump(data, json_file)
        
root_dir = r'C:\Users\andri\PycharmProjects\test04022023'
        
for dirpath, dirnames, filenames in os.walk(root_dir):
    if 'raw_video' in dirnames:
        raw_video_dir = os.path.join(dirpath, 'raw_video')
        for filename in os.listdir(raw_video_dir):
            if filename.endswith('.avi'):
                video_file_path = os.path.join(raw_video_dir, filename)
                try:
                    is_flipped = process_video55(video_file_path)

                    # Cast numpy boolean to native Python boolean
                    is_flipped = bool(is_flipped)
                    json_data = {"is_flipped": is_flipped}

                    if is_flipped:
                        print(f'{filename} is flipped')
                        flipped_video_file_path = flip_video(video_file_path)

                        # Move the original video to the original_video folder
                        original_video_dir = os.path.join(raw_video_dir, "original_video")
                        os.makedirs(original_video_dir, exist_ok=True)
                        os.rename(video_file_path, os.path.join(original_video_dir, filename))

                        # Rename the flipped video to the original file name
                        os.rename(flipped_video_file_path, video_file_path)
                    else:
                        print(f'{filename} is not flipped')

                    # Create a JSON file with is_flipped information
                    json_file_path = os.path.join(raw_video_dir, os.path.splitext(filename)[0] + '.json')
                    create_json_file(json_file_path, json_data)

                except Exception as e:
                    if isinstance(e, (FileNotFoundError, OSError, IOError)):
                        print(f"{filename} does not contain any frames for analysis. Please check the video. Error: {e}")
                    else:
                        print(f"An error occurred while processing {filename}. Error: {e}")